# Genbank phylogeny from COI sequences

Complete mitogenome genbank records were downloaded for 4 ant species, each from a different subfamily:
  - ***Pseudomyrmex gracilis*** (Pseudomyrmecinae)
  - ***Formica fusca*** (Formicinae)
  - ***Linepthema humile*** (Dolichoderinae)
  - ***Solenopsis invicta*** (Myrmicinae)

In [1]:
# importing everything we'll need
from Bio import SeqIO
import os, glob, itertools
import pandas as pd

Now, we'll be using SeqIO to obtain the COI nucleotide sequence for each species and save it into separate files:

In [137]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

def extract_COI(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./coi_seqs/{}_coi.fa".format(species_name)
        create_dir(filename)
        with open(filename, "w") as coi_file:
            for gene in record.features:
                if gene.type in ["CDS"] and gene.qualifiers.get('gene')[0] in ['COX1', 'COI']:
                    header = "{}-{}".format(species_name, gene.qualifiers.get('gene')[0])
                    sequence = gene.location.extract(record.seq) # Mas seq pode ter stop codon truncado
                    if len(sequence) % 3 == 1:
                        sequence += "AA" #Resto 1 - Precisa adicionar 'AA'
                    elif len(sequence) % 3 == 2:
                        sequence += "A" #Resto 2 - Precisa adicionar 'A'
                    coi_file.write(">{}\n{}\n".format(header, sequence))

In [137]:
def extract_CYTB(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./seed_fasta//{}_cytb.fa".format(species_name)
        create_dir(filename)
        with open(filename, "w") as coi_file:
            for gene in record.features:
                if gene.type in ["CDS"] and gene.qualifiers.get('gene')[0] in ['COX1', 'COI']:
                    header = "{}-{}".format(species_name, gene.qualifiers.get('gene')[0])
                    sequence = gene.location.extract(record.seq) # Mas seq pode ter stop codon truncado
                    if len(sequence) % 3 == 1:
                        sequence += "AA" #Resto 1 - Precisa adicionar 'AA'
                    elif len(sequence) % 3 == 2:
                        sequence += "A" #Resto 2 - Precisa adicionar 'A'
                    coi_file.write(">{}\n{}\n".format(header, sequence))

In [138]:
for gb_file in glob.glob("./ant_mitogenomes/*.gb"):
    extract_COI(gb_file)

Running blastn with the COI sequences against NCBI's formicidae sequences:

**OBS:** Need to install [taxdb database](ftp://ftp.ncbi.nlm.nih.gov/blast/db/taxdb.tar.gz).

In [27]:
%%bash

# Extracting taxdb database (if necessary)
taxdb=$(pwd)/taxdb
if [[ ! -d "$taxdb" ]]; then 
    echo "taxdb dir not found. Extracting taxdb database..."
    mkdir $taxdb
    tar -C $taxdb -xaf taxdb.tar.gz
else
    echo "taxdb dir found"
fi

# Setting BLASTDB variable
echo "Setting BLASTDB variable to $taxdb"
export BLASTDB=$taxdb

# Creating directory for blast results (if necessary)
if [[ ! -d "blast_results" ]]; then 
    echo "Creating dirrectory for blast results"
    mkdir blast_results
else
    echo "blast_results directory already created"
fi

# Performing blast searches 
for coi in ./coi_seqs/*; do
    echo "Running blast search for $coi..." && 
    #blastn -query $coi -db ./blast_teste/ant_mito -out ./blast_results/$(basename $coi .fa).blast -outfmt "6 qseqid sseqid staxids sscinames stitle sacc saccver slen sstart send qseq";
    blastn -query $coi -db nr -max_target_seqs 5000 -remote -entrez_query "Formicidae [Organism]" -outfmt "6 qseqid staxids sscinames sacc sseq" -out ./blast_results/$(basename $coi .fa).blast;
done 

taxdb dir found
Setting BLASTDB variable to /home/gabriel/Dropbox/repos/genbank_phylogeny/taxdb
blast_results directory already created
Running blast search for ./coi_seqs/Formica_fusca_coi.fa...
Running blast search for ./coi_seqs/Linepithema_humile_coi.fa...
Running blast search for ./coi_seqs/Pseudomyrmex_gracilis_coi.fa...
Running blast search for ./coi_seqs/Solenopsis_invicta_coi.fa...


**NOTE:** Blast with the `-remote` flag can take quite some time to run. There are at least two ways to avoid this:

1. If hard disk space is not a problem, it is possible to download the entire nt database and run it locally...
2. It is possible to run blast with your sequences [online](https://blast.ncbi.nlm.nih.gov/Blast.cgi) and retrieve the formatted results with blast_formatter

      `blast_formatter -rid NMJ61B4A014 -outfmt "6 qseqid staxids sscinames sacc  sseq" > Solenopsis_inivicta_coi.blast`
      
   However, in this second approach it is necessary to run one blast search against NCBI for each seed sequence. A possible improvement to this pipeline would be the use of a single multifasta file with all seeds as input to a blast online search. The user would get the RID for this run and pass it to the pipeline, which by its turn would output the formatted search using blast_formatter. This search would be put into a dataframe and each seed (qseqid) results analysed individually using multiindexing and/or groupby().

Saving the blast results into dataframes:

In [156]:
def create_dataframe(blast_result):
    return pd.read_csv(blast_result, sep='\t',
               names=['qseqid', 'staxids', 'sscinames', 'sacc', 'sseq']) 
    
def extract_columns_dataframe(df):
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseq']]
    df['sseq'] = df['sseq'].apply(lambda x: x.replace('-', ''))
    #df['sseqlen'] = len(df['sseq'])
    df['sseqlen'] = df.apply(lambda row: len(row.sseq), axis = 1) 
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseqlen', 'sseq']]
    return df
    
#df = extract_columns_dataframe(create_dataframe('blast_results/Formica_fusca_coi_old.blast'))
#print(df)
#df.to_excel("blast.xlsx")

In [157]:
#Testing methods of dataframe
#df[df["staxids"].str.contains(";")].index
#df.index
#df[";" in df.staxids].index

Now that we have the blast results in a dataframe, we can clean it in order to:

-  Remove rows with more than one taxid;
-  Sort dataframe (descending) for both taxid and sseqlen;
-  Keep only one record by taxid (the one with the longest sseqlen)

In [158]:
def clean_dataframe(df):
    clean_df = df.drop(df[df.staxids.str.contains(";")].index) # Removing rows with hybrid sequences (more than one taxid value)
    #clean_df["staxids"] = pd.to_numeric(clean_df["staxids"])
    clean_df = clean_df.sort_values(by=["staxids", "sseqlen"], ascending=False) # Sorting dataframe by taxid and sseqlen (descending) - Guarantees that highest sseqlen will always be the first row for that taxid
    # Printing all rows to check output
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    #    print(clean_df[["sscinames", "sacc", "staxids", "sseqlen"]])
    clean_df = clean_df.drop_duplicates(subset="staxids", keep='first') # Keeps only one record per txid. The one that has the highest sseqlen
    return clean_df

#print(clean_dataframe(df).dtypes)
#
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#    print(clean_dataframe(df)[["sscinames", "sacc", "staxids", "sseqlen"]])

Now that we have the functions to extract and clean the data, we have to concatenate the blast results into a single, final dataframe:

In [159]:
blast_data = []
for blast_result in glob.glob("./blast_results/*.blast"):
    blast_data.append(clean_dataframe(extract_columns_dataframe(create_dataframe(blast_result))))
blast_alldata = pd.concat(blast_data)

Despite the warnings, the resulting dataframe is correctly formatted and henceforth suitable for downstream analyses:

In [160]:
blast_alldata

,qseqid,staxids,sscinames,sacc,sseqlen,sseq
2332,Solenopsis_invicta-COX1,997804,Pheidole sp. BOLD:AAC2903,DQ176140,656,ATATTATATTTTATTTTTGCCATTTGGGCTGGAATAATTGGTTCTT...
2640,Solenopsis_invicta-COX1,997764,Pheidole sp. MGs128,HM419549,645,TACTTTATCTTTGCAATCTGATCTGGGATAATTGGATCTTCTATGA...
1534,Solenopsis_invicta-COX1,997694,Pheidole sp. MGs046,DQ176053,657,ATATTATATTTTATTTTTGCAATTTGAGCAGGAATAATTGGATCTT...
4259,Solenopsis_invicta-COX1,997673,Camponotus sp. MG050,HM373109,657,ATTCTATACTTTATTTTTGCAATTTGATCAGGTCTAATTGGCTCAT...
3788,Solenopsis_invicta-COX1,997616,Camponotus sp. MG024,HM373064,657,AATATTATATTTTATTTTTGCAATTTGATCAGGAATAATTGGCTCT...
...,...,...,...,...,...,...
3649,Formica_fusca-COX1,104421,Camponotus floridanus,AY334397,1190,ATTTTATTAACGAAGGATCGGGTACAGGTTGAACTGTTTACCCGCC...
666,Formica_fusca-COX1,1037925,Formica sp. BOLD:AAA1470,KC502375,658,TATTCTTTATTTCTTATTTGCTATTTGAGCAGGAATAATTGGATCT...
512,Formica_fusca-COX1,1037924,Formica sp. BOLD:AAA1469,JX829423,658,TATTCTTTACTTTTTATTTGCTATTTGAGCAGGAATAATTGGATCT...
113,Formica_fusca-COX1,1031672,Messor minor x Messor cf. wasmanni BCSS-2011,EU441259,1203,AATTCATTAATTAACAATGATCAAATTTATAATACTTTAGTGACAA...


Let's create a matrix with the percentage of hits shared between all sequences:

In [161]:
# First, let's create a dict with query headers as keys and a list of subject accession as values:
blast_dict = {k: f.tolist()
     for k, f in blast_alldata.groupby('qseqid')['sacc']}

print(blast_dict)

{'Formica_fusca-COX1': ['HM144377', 'HM144372', 'HM144355', 'HM144359', 'DQ074325', 'AB185475', 'AB019425', 'KT266831', 'KT184578', 'GQ255194', 'GQ255162', 'GQ255191', 'GQ255164', 'GQ255205', 'GQ255201', 'GQ255199', 'AB010937', 'AB010934', 'DQ353343', 'AB010936', 'AB010933', 'AB010932', 'AB010931', 'AB010930', 'AB103360', 'AB103357', 'AB103364', 'AB010926', 'LN607805', 'FJ982440', 'FJ982483', 'FJ982473', 'FJ982472', 'FJ982459', 'FJ982452', 'FJ982451', 'FJ982446', 'FJ982464', 'FJ982462', 'FJ982456', 'GQ255168', 'AB007981', 'NC_049861', 'AB371006', 'KX146469', 'MF134120', 'KC921199', 'KX219960', 'JN562438', 'KU504914', 'HM144379', 'KJ141877', 'HM144368', 'KJ141903', 'KJ141917', 'KJ141821', 'KJ141822', 'HM144350', 'KJ141815', 'KJ141918', 'KJ141886', 'KJ141880', 'KJ141824', 'KJ141817', 'FJ982465', 'FJ982461', 'FJ982458', 'KY770018', 'FJ982469', 'FJ982450', 'FJ982447', 'FJ982442', 'KU504898', 'KU504892', 'GQ255208', 'GQ255193', 'GQ255189', 'GQ255192', 'GU339338', 'GQ255176', 'GQ255173', 'GQ

In [162]:
#Calculating the number of subject accessions shared between two species

def matrix_absolute_matches(blast_dict):
    matrix_dict = {k: [] for k in blast_dict.keys()} #Empty 
    for pair in itertools.product(blast_dict.keys(), repeat=2): #All possible combinations
        match = 0
        for acc in blast_dict[pair[0]]:
            if acc in blast_dict[pair[1]]:
                match += 1
        matrix_dict[pair[0]].append(match)
    return pd.DataFrame(matrix_dict, index=matrix_dict.keys())

# Testing function:

#for pair in itertools.product(blast_dict.keys(), repeat=2): #All possible combinations
#    match = 0
#    for acc in blast_dict[pair[0]]:
#        if acc in blast_dict[pair[1]]:
#            match += 1
#    print("{} x {} = {} identical matches".format(pair[0], pair[1], match))
#
#matrix_absolute_matches(blast_dict)

In [163]:
#Calculating the percentage of subject accessions in species 1 (column) found in species 2 (row)

def matrix_percent_matches(blast_dict):
    matrix_dict = {k: [] for k in blast_dict.keys()} #Empty 
    for pair in itertools.product(blast_dict.keys(), repeat=2): #All possible combinations
        match = 0
        for acc in blast_dict[pair[0]]:
            if acc in blast_dict[pair[1]]:
                match += 1
        matrix_dict[pair[0]].append("{} / {} = {}".format(match, 
                                                          len(blast_dict[pair[0]]), 
                                                              round(match/len(blast_dict[pair[0]]), 2)))
    return pd.DataFrame(matrix_dict, index=matrix_dict.keys())

# Testing function:

#for pair in itertools.product(blast_dict.keys(), repeat=2): #All possible combinations
#    match = 0
#    for acc in blast_dict[pair[0]]:
#        if acc in blast_dict[pair[1]]:
#            match += 1
#    print("{2} percent of the blast hits from {0} are found in {1} ".format(pair[0], pair[1], round(match/len(blast_dict[pair[0]]), 2)))
#
#matrix_percent_matches(blast_dict)

In [174]:
# Printing dataframes (matrices)

matrix_absolute_matches(blast_dict).to_excel("./final_results/matrix_absolute.xlsx")
matrix_absolute_matches(blast_dict)

,Formica_fusca-COX1,Linepithema_humile-COX1,Pseudomyrmex_gracilis-COX1,Solenopsis_invicta-COX1
Formica_fusca-COX1,2718,151,77,130
Linepithema_humile-COX1,151,1556,243,338
Pseudomyrmex_gracilis-COX1,77,243,1176,154
Solenopsis_invicta-COX1,130,338,154,1517


In [175]:
matrix_percent_matches(blast_dict).to_excel("./final_results/matrix_percent.xlsx")
matrix_percent_matches(blast_dict)

,Formica_fusca-COX1,Linepithema_humile-COX1,Pseudomyrmex_gracilis-COX1,Solenopsis_invicta-COX1
Formica_fusca-COX1,2718 / 2718 = 1.0,151 / 1556 = 0.1,77 / 1176 = 0.07,130 / 1517 = 0.09
Linepithema_humile-COX1,151 / 2718 = 0.06,1556 / 1556 = 1.0,243 / 1176 = 0.21,338 / 1517 = 0.22
Pseudomyrmex_gracilis-COX1,77 / 2718 = 0.03,243 / 1556 = 0.16,1176 / 1176 = 1.0,154 / 1517 = 0.1
Solenopsis_invicta-COX1,130 / 2718 = 0.05,338 / 1556 = 0.22,154 / 1176 = 0.13,1517 / 1517 = 1.0


Lastly, let's create 3 final fasta files:

1. With all query sequences
2. With all unique subject sequences from the blast searches
1. With all query + subject sequences

In [167]:
def queries_to_multifasta(directory):
    #create_dir("final_seqs/")
    with open("./final_seqs/queries.fa", "w") as queries:
        for fasta in glob.glob("{}/*fa".format(directory)):
            for record in SeqIO.parse(fasta, "fasta"):
                queries.write(record.format("fasta"))

def subjects_to_multifasta(blast_alldataframe):
    #create_dir("final_seqs/")
    unique_blast_subjects = blast_alldata.sort_values(by=["sseqlen"], ascending=False).drop_duplicates(subset="staxids", keep='first') # Keep only largest sequence per taxid
    with open("./final_seqs/subjects.fa", "w") as subjects:
        for row in unique_blast_subjects.itertuples(): # Awfully slow. Need to optimize this later
            subjects.write(">{}_{}\n{}\n".format(row.sscinames.replace(" ", "_"), row.sacc, row.sseq))
        #unique_blast_subjects.apply(lambda x: subjects.write(">{}_{}\n{}".format(x['sscinames'].replace(" ", "_"), x["sacc"], x['sseq'])))

In [168]:
create_dir("final_seqs/")
queries_to_multifasta("./coi_seqs")
subjects_to_multifasta(blast_alldata)

In [169]:
# Concatenating both multifastas:
!cat final_seqs/queries.fa final_seqs/subjects.fa > final_seqs/all_seqs.fa

That's pretty much it!!! :)

## Complementary functions

In [120]:
# OBS: Using 'staxids' when removing duplicates generates more unique values than 'sacc'

def check_df_len_uniqcol(dataframe):
    print("Length: {}".format(len(dataframe)))
    for column in dataframe.columns:
        print("{} is unique: {}".format(column, dataframe[column].is_unique))
    print()
        
unique_blast_subjects_staxids = blast_alldata.sort_values(by=["sseqlen"], ascending=False).drop_duplicates(subset="staxids", keep='first')
unique_blast_subjects_sacc = blast_alldata.sort_values(by=["sseqlen"], ascending=False).drop_duplicates(subset="sacc", keep='first')
check_df_len_uniqcol(unique_blast_subjects_staxids)
check_df_len_uniqcol(unique_blast_subjects_sacc)

Length: 441
qseqid is unique: False
staxids is unique: True
sscinames is unique: True
sacc is unique: True
sseqlen is unique: False
sseq is unique: True

Length: 473
qseqid is unique: False
staxids is unique: False
sscinames is unique: False
sacc is unique: True
sseqlen is unique: False
sseq is unique: True



In [145]:
# Counting number of columns of blast files
def count_tab_columns_for_each_line(filename, delimiter):
    with open(filename) as fh:
        for linenum, line in enumerate(fh, start=1):
            print("Line {}: {} columns".format(linenum, len(line.split(delimiter))))

In [171]:
count_tab_columns_for_each_line("./blast_results/Pseudomyrmex_gracilis_coi.blast", "\t")

Line 1: 5 columns
Line 2: 5 columns
Line 3: 5 columns
Line 4: 5 columns
Line 5: 5 columns
Line 6: 5 columns
Line 7: 5 columns
Line 8: 5 columns
Line 9: 5 columns
Line 10: 5 columns
Line 11: 5 columns
Line 12: 5 columns
Line 13: 5 columns
Line 14: 5 columns
Line 15: 5 columns
Line 16: 5 columns
Line 17: 5 columns
Line 18: 5 columns
Line 19: 5 columns
Line 20: 5 columns
Line 21: 5 columns
Line 22: 5 columns
Line 23: 5 columns
Line 24: 5 columns
Line 25: 5 columns
Line 26: 5 columns
Line 27: 5 columns
Line 28: 5 columns
Line 29: 5 columns
Line 30: 5 columns
Line 31: 5 columns
Line 32: 5 columns
Line 33: 5 columns
Line 34: 5 columns
Line 35: 5 columns
Line 36: 5 columns
Line 37: 5 columns
Line 38: 5 columns
Line 39: 5 columns
Line 40: 5 columns
Line 41: 5 columns
Line 42: 5 columns
Line 43: 5 columns
Line 44: 5 columns
Line 45: 5 columns
Line 46: 5 columns
Line 47: 5 columns
Line 48: 5 columns
Line 49: 5 columns
Line 50: 5 columns
Line 51: 5 columns
Line 52: 5 columns
Line 53: 5 columns
Li